# DaltonLens template

* Cells before the `# [[nbplot]] template` are ignored.
* Cells starting with `# [[nbplot]] ignore` are also ignored.

* Some variables are substituted in every cell:
    * `${root_path}`: the working directory when `nbplot` was called. Input files will be relative to this.

* Some variables are subtituted in the `[[nbplot]] for i,input in enumerate(inputs)` blocks:
    * `${i}`: index of the input in the list
    * `${input.pretty_name}`: truncated path of the file, or 'stdin'
    * `${input.rel_path}`: path of the file relative to the `root_path`, or `stdin`
    * `${input.abs_path_or_io}`: full filepath or StringIO when the data comes from stdin
    * `${input.guessed_sep}`: separator guessed by nbplot for this file. Usually space or comma.

In [ ]:
# [[nbplot]] template
# Note: don't change that first line, it tells nbplot that the notebook below is a template
# This cell will be executed and the metadata dictionary loaded, but not included in the output.

template_metadata = {
    'name': 'daltonlens',
    'format_version': '0.1'
}

In [ ]:
import io, math, os, sys
from base64 import b64decode
from pathlib import Path
from IPython.core.display import HTML

import matplotlib.pyplot as plt
import numpy as np

import PIL

# Install daltonlens if necessary
try:
    from daltonlens import convert, simulate
except ImportError:
    %pip install -q daltonlens
    from daltonlens import convert, simulate

# Uncomment to get interactive plots.
# %matplotlib ipympl

simulator = simulate.Simulator_Vienot1999(convert.LMSModel_sRGB_SmithPokorny75())
# simulator = simulate.Simulator_Brettel1997(convert.LMSModel_sRGB_SmithPokorny75())

# Convert a float image to uint8 with proper clipping to 0,255
def image_to_uint8(im): return np.clip(im, 0., 255.).astype(np.uint8)

def process_image(name, abs_path_or_io):
    im = np.asarray(PIL.Image.open(abs_path_or_io).convert('RGB'))

    plt.ioff();
    
    display(HTML('<hr>'))    
    fig,axs = plt.subplots(1, 3, figsize=(16,8), constrained_layout=True)
    np_im = np.array(im)
    np_flip_red_blue = np_im[:,:,[2,1,0]]
    np_flip_green_blue = np_im[:,:,[0,2,1]]
    axs[0].imshow(im)
    axs[0].set_title(name)
    axs[1].imshow(np_flip_red_blue)
    axs[1].set_title('Flip Red/Blue')
    axs[2].imshow(np_flip_green_blue)
    axs[2].set_title('Flip Green/Blue')    
    plt.show()   
    
    display(HTML('<hr>'))
    fig,axs = plt.subplots(1, 3, figsize=(16,8), constrained_layout=True)
    axs[0].imshow(simulator.simulate_cvd (im, simulate.Deficiency.PROTAN, 1.0))
    axs[0].set_title('Protanope simulation')
    axs[1].imshow(simulator.simulate_cvd (im, simulate.Deficiency.DEUTAN, 1.0))
    axs[1].set_title('Deuteranope simulation')
    axs[2].imshow(simulator.simulate_cvd (im, simulate.Deficiency.TRITAN, 1.0))
    axs[2].set_title('Tritanope simulation')
    plt.show()

    
    plt.ion(); # restore interactive mode

In [ ]:
root_path = Path("$root_path")

# [[nbplot]] for i,input in enumerate(inputs)
process_image("${input.pretty_name}", ${input.abs_path_or_io})
# [[nbplot]] endfor